# Route identification (time-series)

Over time, even `route_ids` change. Pick out a couple of examples of this.

In [1]:
import pandas as pd
import yaml

from update_vars import SCHED_GCS, GTFS_DATA_DICT
from shared_utils import portfolio_utils
from segment_speed_utils import time_series_utils

with open(
    "../_shared_utils/shared_utils/portfolio_organization_name.yml", "r"
) as f:
    PORTFOLIO_ORGANIZATIONS_DICT = yaml.safe_load(f)

In [2]:
CLEANED_ROUTE_NAMING = GTFS_DATA_DICT.schedule_tables.route_identification

df = pd.read_parquet(
    f"{SCHED_GCS}{CLEANED_ROUTE_NAMING}.parquet"
).pipe(
    portfolio_utils.standardize_portfolio_organization_names, 
    PORTFOLIO_ORGANIZATIONS_DICT
)

In [3]:
df.columns

Index(['schedule_gtfs_dataset_key', 'name', 'route_id', 'route_long_name',
       'route_short_name', 'route_desc', 'service_date', 'combined_name',
       'route_id2', 'recent_combined_name', 'recent_route_id2',
       'portfolio_organization_name'],
      dtype='object')

## LA Metro

`route_id` has suffix added every time a new feed goes into effect.

In [4]:
subset_cols = [
    "name", "portfolio_organization_name", 
    "route_id", "service_date", "combined_name",
    "recent_combined_name", "recent_route_id2", 'route_long_name',
       'route_short_name',
]

df[(df.name.str.contains("LA Metro")) & 
   (df.recent_combined_name == "2__Metro Local Line")
][subset_cols].tail(10)

,name,portfolio_organization_name,route_id,service_date,combined_name,recent_combined_name,recent_route_id2,route_long_name,route_short_name
99231,LA Metro Bus Schedule,Los Angeles County Metropolitan Transportation...,2-13183,2024-10-18,2__Metro Local Line,2__Metro Local Line,2,Metro Local Line,2
101311,LA Metro Bus Schedule,Los Angeles County Metropolitan Transportation...,2-13183,2024-10-19,2__Metro Local Line,2__Metro Local Line,2,Metro Local Line,2
102996,LA Metro Bus Schedule,Los Angeles County Metropolitan Transportation...,2-13183,2024-10-20,2__Metro Local Line,2__Metro Local Line,2,Metro Local Line,2
105242,LA Metro Bus Schedule,Los Angeles County Metropolitan Transportation...,2-13183,2024-11-13,2__Metro Local Line,2__Metro Local Line,2,Metro Local Line,2
108035,LA Metro Bus Schedule,Los Angeles County Metropolitan Transportation...,2-13183,2024-12-11,2__Metro Local Line,2__Metro Local Line,2,Metro Local Line,2
111088,LA Metro Bus Schedule,Los Angeles County Metropolitan Transportation...,2-13186,2025-01-15,2__Metro Local Line,2__Metro Local Line,2,Metro Local Line,2
113814,LA Metro Bus Schedule,Los Angeles County Metropolitan Transportation...,2-13191,2025-02-12,2__Metro Local Line,2__Metro Local Line,2,Metro Local Line,2
116376,LA Metro Bus Schedule,Los Angeles County Metropolitan Transportation...,2-13191,2025-03-12,2__Metro Local Line,2__Metro Local Line,2,Metro Local Line,2
119065,LA Metro Bus Schedule,Los Angeles County Metropolitan Transportation...,2-13191,2025-04-16,2__Metro Local Line,2__Metro Local Line,2,Metro Local Line,2
121607,LA Metro Bus Schedule,Los Angeles County Metropolitan Transportation...,2-13191,2025-05-14,2__Metro Local Line,2__Metro Local Line,2,Metro Local Line,2


In [5]:
df[(df.name.str.contains("LA Metro")) & 
   (df.recent_combined_name == "2__Metro Local Line")
  ].route_id.value_counts()

2-13172    13
2-13183    12
2-13168    11
2-13167    10
2-13191     4
2-13186     1
Name: route_id, dtype: int64

## VCTC

These were flagged as a complicated case where metrics were duplicated in GTFS Digest.

Within [time_series_utils](https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/segment_speed_utils/time_series_utils.py#L84-L105), in Apr 2024, when this function was added, VCTC should only have route_long_names kept. 

Recently, since Sep 2024, it appears they've grouped a set of routes together, now it's appearing as 80-89 Coastal Express.
Given that, we should remove VCTC GMV from that list where we do extra route cleaning.

In [6]:
vctc_df = df[(df.name=="VCTC GMV Schedule") & 
   (df.recent_combined_name.str.contains("Coastal Express"))
  ]
vctc_df[subset_cols]

,name,portfolio_organization_name,route_id,service_date,combined_name,recent_combined_name,recent_route_id2,route_long_name,route_short_name
14,VCTC GMV Schedule,"Ventura County (VCTC, Gold Coast, Cities of Ca...",4134,2023-03-15,80__80-Coastal Express,80__80-Coastal Express,4134,80-Coastal Express,80
15,VCTC GMV Schedule,"Ventura County (VCTC, Gold Coast, Cities of Ca...",4136,2023-03-15,80X__80x-Coastal Express,80X__80x-Coastal Express,4136,80x-Coastal Express,80X
16,VCTC GMV Schedule,"Ventura County (VCTC, Gold Coast, Cities of Ca...",4145,2023-03-15,86__86-Coastal Express,86__86-Coastal Express,4145,86-Coastal Express,86
21,VCTC GMV Schedule,"Ventura County (VCTC, Gold Coast, Cities of Ca...",4143,2023-03-15,85__85-Coastal Express,85__85-Coastal Express,4143,85-Coastal Express,85
22,VCTC GMV Schedule,"Ventura County (VCTC, Gold Coast, Cities of Ca...",4146,2023-03-15,87__87-Coastal Express,87__87-Coastal Express,4146,87-Coastal Express,87
...,...,...,...,...,...,...,...,...,...
121729,VCTC GMV Schedule,"Ventura County (VCTC, Gold Coast, Cities of Ca...",4146,2025-05-14,87__80-89 Coastal Express,87__80-89 Coastal Express,4146,80-89 Coastal Express,87
121818,VCTC GMV Schedule,"Ventura County (VCTC, Gold Coast, Cities of Ca...",4136,2025-05-14,80X__80-89 Coastal Express,80X__80-89 Coastal Express,4136,80-89 Coastal Express,80X
121821,VCTC GMV Schedule,"Ventura County (VCTC, Gold Coast, Cities of Ca...",4138,2025-05-14,81B__80-89 Coastal Express,81B__80-89 Coastal Express,4138,80-89 Coastal Express,81B
121822,VCTC GMV Schedule,"Ventura County (VCTC, Gold Coast, Cities of Ca...",4142,2025-05-14,84U__80-89 Coastal Express,84U__80-89 Coastal Express,4142,80-89 Coastal Express,84U


In [7]:
vctc_df.pipe(
    time_series_utils.clean_standardized_route_names
)

,schedule_gtfs_dataset_key,name,route_id,route_long_name,route_short_name,route_desc,service_date,combined_name,route_id2,recent_combined_name,recent_route_id2,portfolio_organization_name
0,7e015887964432c82ce7e735c2753f86,VCTC GMV Schedule,4134,80-Coastal Express,80,80,2023-03-15,80__80-Coastal Express,4134,80-Coastal Express,4134,"Ventura County (VCTC, Gold Coast, Cities of Ca..."
1,7e015887964432c82ce7e735c2753f86,VCTC GMV Schedule,4136,80x-Coastal Express,80X,80x,2023-03-15,80X__80x-Coastal Express,4136,80x-Coastal Express,4136,"Ventura County (VCTC, Gold Coast, Cities of Ca..."
2,7e015887964432c82ce7e735c2753f86,VCTC GMV Schedule,4145,86-Coastal Express,86,86,2023-03-15,86__86-Coastal Express,4145,86-Coastal Express,4145,"Ventura County (VCTC, Gold Coast, Cities of Ca..."
3,7e015887964432c82ce7e735c2753f86,VCTC GMV Schedule,4143,85-Coastal Express,85,85,2023-03-15,85__85-Coastal Express,4143,85-Coastal Express,4143,"Ventura County (VCTC, Gold Coast, Cities of Ca..."
4,7e015887964432c82ce7e735c2753f86,VCTC GMV Schedule,4146,87-Coastal Express,87,87,2023-03-15,87__87-Coastal Express,4146,87-Coastal Express,4146,"Ventura County (VCTC, Gold Coast, Cities of Ca..."
...,...,...,...,...,...,...,...,...,...,...,...,...
529,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,4146,80-89 Coastal Express,87,87,2025-05-14,87__80-89 Coastal Express,4146,80-89 Coastal Express,4146,"Ventura County (VCTC, Gold Coast, Cities of Ca..."
530,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,4136,80-89 Coastal Express,80X,80x,2025-05-14,80X__80-89 Coastal Express,4136,80-89 Coastal Express,4136,"Ventura County (VCTC, Gold Coast, Cities of Ca..."
531,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,4138,80-89 Coastal Express,81B,81B,2025-05-14,81B__80-89 Coastal Express,4138,80-89 Coastal Express,4138,"Ventura County (VCTC, Gold Coast, Cities of Ca..."
532,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,4142,80-89 Coastal Express,84U,84U,2025-05-14,84U__80-89 Coastal Express,4142,80-89 Coastal Express,4142,"Ventura County (VCTC, Gold Coast, Cities of Ca..."


In [8]:
unique_routes = vctc_df.pipe(
    time_series_utils.clean_standardized_route_names
).route_id.unique()

The new `recent_combined_name` since Sep 2024-May 2025 is capturing so many route_ids!

This confirms what was seen GTFS Digest.

In [9]:
vctc_df.pipe(
    time_series_utils.clean_standardized_route_names
).astype(str).groupby(
    ["recent_combined_name"]
).agg({
    "route_id": lambda x: list(set(x)),
    "service_date": ["min", "max"],
}).reset_index()

recent_combined_name                                           route_id  \
                                                                    <lambda>   
0   80-89 Coastal Express  [4146, 4138, 4134, 4147, 4142, 4141, 4137, 414...   
1      80-Coastal Express                                             [4134]   
2     80c-Coastal Express                                             [4135]   
3     80x-Coastal Express                                             [4136]   
4      81-Coastal Express                                             [4137]   
5     81B-Coastal Express                                             [4138]   
6      84-Coastal Express                                             [4141]   
7     84u-Coastal Express                                             [4142]   
8      85-Coastal Express                                             [4143]   
9     85c-Coastal Express                                             [4144]   
10     86-Coastal Express                                             [4145]   
11     87-Coastal Express                                             [4146]   
12     88-Coastal Express                                             [4147]   
13     89-Coastal Express                                             [4148]   
14        Coastal Express  [4146, 4138, 4134, 4147, 4142, 4141, 4137, 414...   

   service_date              
            min         max  
0    2024-09-18  2025-05-14  
1    2023-03-15  2024-06-12  
2    2023-03-15  2023-08-15  
3    2023-03-15  2024-06-12  
4    2023-03-15  2024-06-12  
5    2023-03-15  2024-06-12  
6    2023-03-15  2024-06-12  
7    2023-03-15  2024-06-12  
8    2023-03-15  2024-06-12  
9    2023-03-15  2024-06-12  
10   2023-03-15  2024-06-12  
11   2023-03-15  2024-06-12  
12   2023-03-15  2024-06-12  
13   2023-03-15  2024-06-12  
14   2024-07-17  2024-08-14

In Apr 2024, this looks ok, `recent_combined_name` is reasonable.

In [10]:
vctc_df[vctc_df.service_date <= pd.to_datetime("2024-04-01")].pipe(
    time_series_utils.clean_standardized_route_names
).astype(str).groupby(
    ["recent_combined_name"]
).agg({
    "route_id": lambda x: list(set(x)),
    "service_date": ["min", "max"],
}).reset_index()

recent_combined_name route_id service_date            
                        <lambda>          min         max
0    80-Coastal Express   [4134]   2023-03-15  2024-03-13
1   80c-Coastal Express   [4135]   2023-03-15  2023-08-15
2   80x-Coastal Express   [4136]   2023-03-15  2024-03-13
3    81-Coastal Express   [4137]   2023-03-15  2024-03-13
4   81B-Coastal Express   [4138]   2023-03-15  2024-03-13
5    84-Coastal Express   [4141]   2023-03-15  2024-03-13
6   84u-Coastal Express   [4142]   2023-03-15  2024-03-13
7    85-Coastal Express   [4143]   2023-03-15  2024-03-13
8   85c-Coastal Express   [4144]   2023-03-15  2024-03-13
9    86-Coastal Express   [4145]   2023-03-15  2024-03-13
10   87-Coastal Express   [4146]   2023-03-15  2024-03-13
11   88-Coastal Express   [4147]   2023-03-15  2024-03-13
12   89-Coastal Express   [4148]   2023-03-15  2024-03-13

In [11]:
def display_subset(df: pd.DataFrame) -> tuple[pd.DataFrame]:
    """
    Compare what happens when we pipe vs don't pipe for the operators in the list we
    do extra data wrangling on.
    """
    cols = ["route_id", "route_short_name", "route_long_name", 
         "recent_combined_name"
           ]
    with_pipe = df.pipe(
        time_series_utils.clean_standardized_route_names
    ).astype(str).groupby(
        ["recent_combined_name"]
    ).agg({
        "route_id": lambda x: list(set(x)),
        "service_date": ["min", "max"],
    }).reset_index()

    no_pipe = df.astype(str).groupby(
        ["recent_combined_name"]
    ).agg({
        "route_id": lambda x: list(set(x)),
        "service_date": ["min", "max"],
    }).reset_index()
    
    print(f"status quo, with time_series_utils.pipe affecting it")
    display(with_pipe)
    
    print(f"status quo, no pipe, no extra parsing affecting it")
    display(no_pipe)
    
    return

In [12]:
display_subset(vctc_df)

status quo, with time_series_utils.pipe affecting it


recent_combined_name                                           route_id  \
                                                                    <lambda>   
0   80-89 Coastal Express  [4146, 4138, 4134, 4147, 4142, 4141, 4137, 414...   
1      80-Coastal Express                                             [4134]   
2     80c-Coastal Express                                             [4135]   
3     80x-Coastal Express                                             [4136]   
4      81-Coastal Express                                             [4137]   
5     81B-Coastal Express                                             [4138]   
6      84-Coastal Express                                             [4141]   
7     84u-Coastal Express                                             [4142]   
8      85-Coastal Express                                             [4143]   
9     85c-Coastal Express                                             [4144]   
10     86-Coastal Express                                             [4145]   
11     87-Coastal Express                                             [4146]   
12     88-Coastal Express                                             [4147]   
13     89-Coastal Express                                             [4148]   
14        Coastal Express  [4146, 4138, 4134, 4147, 4142, 4141, 4137, 414...   

   service_date              
            min         max  
0    2024-09-18  2025-05-14  
1    2023-03-15  2024-06-12  
2    2023-03-15  2023-08-15  
3    2023-03-15  2024-06-12  
4    2023-03-15  2024-06-12  
5    2023-03-15  2024-06-12  
6    2023-03-15  2024-06-12  
7    2023-03-15  2024-06-12  
8    2023-03-15  2024-06-12  
9    2023-03-15  2024-06-12  
10   2023-03-15  2024-06-12  
11   2023-03-15  2024-06-12  
12   2023-03-15  2024-06-12  
13   2023-03-15  2024-06-12  
14   2024-07-17  2024-08-14

status quo, no pipe, no extra parsing affecting it


recent_combined_name route_id service_date            
                               <lambda>          min         max
0     80C__80c-Coastal Express   [4135]   2023-03-15  2023-08-15
1   80X__80-89 Coastal Express   [4136]   2023-04-10  2025-05-14
2     80X__80x-Coastal Express   [4136]   2023-03-15  2023-03-15
3    80__80-89 Coastal Express   [4134]   2023-04-10  2025-05-14
4       80__80-Coastal Express   [4134]   2023-03-15  2023-03-15
5   81B__80-89 Coastal Express   [4138]   2023-04-10  2025-05-14
6     81B__81B-Coastal Express   [4138]   2023-03-15  2023-03-15
7    81__80-89 Coastal Express   [4137]   2023-04-10  2025-05-14
8       81__81-Coastal Express   [4137]   2023-03-15  2023-03-15
9   84U__80-89 Coastal Express   [4142]   2023-04-10  2025-05-14
10    84U__84u-Coastal Express   [4142]   2023-03-15  2023-03-15
11   84__80-89 Coastal Express   [4141]   2023-04-10  2025-05-14
12      84__84-Coastal Express   [4141]   2023-03-15  2023-03-15
13  85C__80-89 Coastal Express   [4144]   2023-04-10  2025-05-14
14    85C__85c-Coastal Express   [4144]   2023-03-15  2023-03-15
15   85__80-89 Coastal Express   [4143]   2023-04-10  2025-05-14
16      85__85-Coastal Express   [4143]   2023-03-15  2023-03-15
17   86__80-89 Coastal Express   [4145]   2023-04-10  2025-05-14
18      86__86-Coastal Express   [4145]   2023-03-15  2023-03-15
19   87__80-89 Coastal Express   [4146]   2023-04-10  2025-05-14
20      87__87-Coastal Express   [4146]   2023-03-15  2023-03-15
21   88__80-89 Coastal Express   [4147]   2023-04-10  2025-05-14
22      88__88-Coastal Express   [4147]   2023-03-15  2023-03-15
23   89__80-89 Coastal Express   [4148]   2023-04-10  2025-05-14
24      89__89-Coastal Express   [4148]   2023-03-15  2023-03-15

## Check other operators

Find other operators ones that might also have had changes and see if any can benefit from having extra parsing removed.

In [13]:
for operator in time_series_utils.operators_only_route_long_name:
    print(f"Operator: {operator}")
          
    display_subset(df[df.name==operator])

Operator: Antelope Valley Transit Authority Schedule
status quo, with time_series_utils.pipe affecting it


recent_combined_name route_id service_date            
                        <lambda>          min         max
0               Route 1      [1]   2023-03-15  2025-05-14
1              Route 11     [11]   2023-03-15  2025-05-14
2              Route 12     [12]   2023-03-15  2025-05-14
3               Route 2      [2]   2023-03-15  2025-05-14
4               Route 3      [3]   2023-03-15  2025-05-14
5               Route 4      [4]   2023-03-15  2025-05-14
6               Route 5      [5]   2023-03-15  2025-05-14
7              Route 50     [50]   2023-03-15  2025-05-14
8              Route 51     [51]   2023-03-15  2025-05-14
9              Route 52     [52]   2023-03-15  2025-05-14
10              Route 7      [7]   2023-03-15  2025-05-14
11            Route 785    [785]   2023-03-15  2025-05-14
12            Route 786    [786]   2023-03-15  2025-05-14
13            Route 787    [787]   2023-03-15  2025-05-14
14            Route 790    [790]   2023-03-15  2025-05-14
15              Route 8      [8]   2023-03-15  2025-05-14
16              Route 9      [9]   2023-03-15  2025-05-14
17             Route 94     [94]   2023-03-15  2025-05-14
18             Route 97     [97]   2023-03-15  2025-05-14
19             Route 98     [98]   2023-03-15  2025-05-14

status quo, no pipe, no extra parsing affecting it


recent_combined_name route_id service_date            
                        <lambda>          min         max
0          11__Route 11     [11]   2023-03-15  2025-05-14
1          12__Route 12     [12]   2023-03-15  2025-05-14
2            1__Route 1      [1]   2023-03-15  2025-05-14
3            2__Route 2      [2]   2023-03-15  2025-05-14
4            3__Route 3      [3]   2023-03-15  2025-05-14
5            4__Route 4      [4]   2023-03-15  2025-05-14
6          50__Route 50     [50]   2023-03-15  2025-05-14
7          51__Route 51     [51]   2023-03-15  2025-05-14
8          52__Route 52     [52]   2023-03-15  2025-05-14
9            5__Route 5      [5]   2023-03-15  2025-05-14
10       785__Route 785    [785]   2023-03-15  2025-05-14
11       786__Route 786    [786]   2023-03-15  2025-05-14
12       787__Route 787    [787]   2023-03-15  2025-05-14
13       790__Route 790    [790]   2023-03-15  2025-05-14
14           7__Route 7      [7]   2023-03-15  2025-05-14
15           8__Route 8      [8]   2023-03-15  2025-05-14
16         94__Route 94     [94]   2023-03-15  2025-05-14
17         97__Route 97     [97]   2023-03-15  2025-05-14
18         98__Route 98     [98]   2023-03-15  2025-05-14
19           9__Route 9      [9]   2023-03-15  2025-05-14

Operator: Bay Area 511 ACE Schedule
status quo, with time_series_utils.pipe affecting it


...

recent_combined_name route_id service_date            
                        <lambda>          min         max
0     Rt 11X__Route 11X   [2041]   2023-03-15  2025-05-14
1       Rt 12__Route 12   [2043]   2023-03-15  2025-05-14
2       Rt 15__Route 15   [2040]   2023-03-15  2025-05-14
3       Rt 16__Route 16   [2144]   2023-03-15  2025-05-14
4       Rt 17__Route 17   [5597]   2023-03-15  2025-05-14
5         Rt 1__Route 1   [2037]   2023-03-15  2025-05-14
6         Rt 2__Route 2   [2035]   2023-03-15  2025-05-14
7         Rt 4__Route 4   [2036]   2023-03-15  2025-05-14
8         Rt 5__Route 5   [2032]   2023-03-15  2025-05-14
9         Rt 6__Route 6   [2031]   2023-03-15  2025-05-14
10        Rt 7__Route 7   [4769]   2023-03-15  2025-05-14
11        Rt 8__Route 8   [4772]   2023-03-15  2025-05-14
12        Rt 9__Route 9   [2042]   2023-03-15  2025-05-14

Operator: Yolobus Schedule
status quo, with time_series_utils.pipe affecting it


recent_combined_name                                           route_id  \
                                                                  <lambda>   
0   Causeway Connection  [b6bc2958-333f-4d1e-8579-b75b2698b6b4, 9cf7725...   
1             ROUTE 211  [6ffe5400-193c-45b8-856a-e2316995d101, 6ebedf5...   
2             ROUTE 212  [e89fbda1-8475-4442-88ca-852d78d62e46, 3997325...   
3             ROUTE 215                                              [215]   
4          ROUTE 215 EB  [1ae28e33-41d5-41b3-a481-9f51490ec40e, 3bf6177...   
5          ROUTE 215 WB  [f92eae86-583c-4e18-8d51-bf7a435105cb, 3b5dff1...   
6             ROUTE 230                                              [230]   
7          ROUTE 230 AM  [ddf3b183-1d3a-4b20-89f1-0cca1119fc8c, 45abada...   
8          ROUTE 230 PM  [5a402a5e-1527-472a-bbe3-9ca39ef55293, 60bc879...   
9             ROUTE 240  [12a618d6-d441-4f89-be83-aa402b867ed8, 2b5c285...   
10             ROUTE 37  [9831f8e4-7142-4d62-8a86-2cf3ee52eae3, 178155d...   
11             ROUTE 40  [8bdcf7fa-0f8b-46c5-aa03-9c590f06ee93, 40, f88...   
12             ROUTE 41  [41, a76f7f34-9287-4ce9-bda1-7fdc66130e09, 332...   
13            ROUTE 42A  [08a6c620-4d4c-4be5-8dd9-9172952a13d8, fc37bce...   
14            ROUTE 42B  [c51c649f-0092-4961-919f-01e1afd0625f, 443c277...   
15             ROUTE 43                                               [43]   
16          ROUTE 43 AM  [f51d601d-4df7-4054-8c39-b44536d9c734, cc65411...   
17          ROUTE 43 PM  [ac8ab6d7-5799-4b80-bf7a-41b140cdc705, c1b5d90...   
18           ROUTE 43-R                                              [43R]   
19        ROUTE 43-R AM  [43RAM, af92ad39-d491-4c92-adff-2bf4a1de91fd, ...   
20         ROUTE 43R PM  [43RPM, f12acfcf-4456-43dd-9749-4a01514ba209, ...   
21             ROUTE 44                                               [44]   
22             ROUTE 45                                               [45]   
23          ROUTE 45 AM  [398bc159-24d1-4c5a-b0b3-501871c77335, 3e2bcbf...   
24          ROUTE 45 PM  [235c6366-5e82-4a56-8fb0-f574bd72c23c, 45PM, 2...   

   service_date              
            min         max  
0    2023-03-15  2025-05-14  
1    2023-03-15  2025-05-14  
2    2023-03-15  2025-05-14  
3    2024-07-17  2025-05-14  
4    2023-03-15  2024-06-12  
5    2023-03-15  2024-06-12  
6    2024-07-17  2025-05-14  
7    2023-03-15  2024-06-12  
8    2023-03-15  2024-06-12  
9    2023-03-15  2025-05-14  
10   2023-03-15  2025-05-14  
11   2023-03-15  2025-05-14  
12   2023-03-15  2025-05-14  
13   2023-03-15  2025-05-14  
14   2023-03-15  2025-05-14  
15   2025-03-12  2025-05-14  
16   2023-03-15  2025-02-12  
17   2023-03-15  2025-02-12  
18   2025-03-12  2025-05-14  
19   2023-03-15  2025-02-12  
20   2023-03-15  2025-02-12  
21   2025-02-12  2025-05-14  
22   2025-02-12  2025-05-14  
23   2023-03-15  2025-01-15  
24   2023-03-15  2025-01-15

status quo, no pipe, no extra parsing affecting it


recent_combined_name  \
                               
0   138__Causeway Connection   
1             211__ROUTE 211   
2             212__ROUTE 212   
3        215EB__ROUTE 215 EB   
4        215WB__ROUTE 215 WB   
5             215__ROUTE 215   
6        230AM__ROUTE 230 AM   
7        230PM__ROUTE 230 PM   
8             230__ROUTE 230   
9             240__ROUTE 240   
10              37__ROUTE 37   
11              40__ROUTE 40   
12              41__ROUTE 41   
13            42A__ROUTE 42A   
14            42B__ROUTE 42B   
15         43AM__ROUTE 43 AM   
16         43PM__ROUTE 43 PM   
17      43RAM__ROUTE 43-R AM   
18       43RPM__ROUTE 43R PM   
19           43R__ROUTE 43-R   
20              43__ROUTE 43   
21              44__ROUTE 44   
22         45AM__ROUTE 45 AM   
23         45PM__ROUTE 45 PM   
24              45__ROUTE 45   

                                             route_id service_date              
                                             <lambda>          min         max  
0   [b6bc2958-333f-4d1e-8579-b75b2698b6b4, 9cf7725...   2023-03-15  2025-05-14  
1   [6ffe5400-193c-45b8-856a-e2316995d101, 6ebedf5...   2023-03-15  2025-05-14  
2   [e89fbda1-8475-4442-88ca-852d78d62e46, 3997325...   2023-03-15  2025-05-14  
3   [1ae28e33-41d5-41b3-a481-9f51490ec40e, 3bf6177...   2023-03-15  2024-06-12  
4   [f92eae86-583c-4e18-8d51-bf7a435105cb, 3b5dff1...   2023-03-15  2024-06-12  
5                                               [215]   2024-07-17  2025-05-14  
6   [ddf3b183-1d3a-4b20-89f1-0cca1119fc8c, 45abada...   2023-03-15  2024-06-12  
7   [5a402a5e-1527-472a-bbe3-9ca39ef55293, 60bc879...   2023-03-15  2024-06-12  
8                                               [230]   2024-07-17  2025-05-14  
9   [12a618d6-d441-4f89-be83-aa402b867ed8, 2b5c285...   2023-03-15  2025-05-14  
10  [9831f8e4-7142-4d62-8a86-2cf3ee52eae3, 178155d...   2023-03-15  2025-05-14  
11  [8bdcf7fa-0f8b-46c5-aa03-9c590f06ee93, 40, f88...   2023-03-15  2025-05-14  
12  [41, a76f7f34-9287-4ce9-bda1-7fdc66130e09, 332...   2023-03-15  2025-05-14  
13  [08a6c620-4d4c-4be5-8dd9-9172952a13d8, fc37bce...   2023-03-15  2025-05-14  
14  [c51c649f-0092-4961-919f-01e1afd0625f, 443c277...   2023-03-15  2025-05-14  
15  [f51d601d-4df7-4054-8c39-b44536d9c734, cc65411...   2023-03-15  2025-02-12  
16  [ac8ab6d7-5799-4b80-bf7a-41b140cdc705, c1b5d90...   2023-03-15  2025-02-12  
17  [43RAM, af92ad39-d491-4c92-adff-2bf4a1de91fd, ...   2023-03-15  2025-02-12  
18  [43RPM, f12acfcf-4456-43dd-9749-4a01514ba209, ...   2023-03-15  2025-02-12  
19                                              [43R]   2025-03-12  2025-05-14  
20                                               [43]   2025-03-12  2025-05-14  
21                                               [44]   2025-02-12  2025-05-14  
22  [398bc159-24d1-4c5a-b0b3-501871c77335, 3e2bcbf...   2023-03-15  2025-01-15  
23  [235c6366-5e82-4a56-8fb0-f574bd72c23c, 45PM, 2...   2023-03-15  2025-01-15  
24                                               [45]   2025-02-12  2025-05-14